# Project 4: Wrangle and Analyze Data
## Wrangle Act Part 1: Gathering, Assessing and Cleaning Data
### Introduction

__Your tasks in this project are as follows:__

Data wrangling, which consists of:
  * Gathering data (downloadable file in the Resources tab in the left most panel of your classroom and linked in step 1 below).
  * Assessing data
  * Cleaning data
  * Storing, analyzing, and visualizing your wrangled data
  * Reporting on 1) your data wrangling efforts and 2) your data analyses and visualizations


__Key points to keep in mind when data wrangling for this project:__

  * You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
  * Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
  * Cleaning includes merging individual pieces of data according to the rules of tidy data.
  * The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.
  * You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.


## Environment Preparation

In [420]:
import pandas as pd
import numpy as np
import getapi
import json
import tweepy
import requests
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

## Data Gathering
### Open We Rate Dogs Archive

In [157]:
wrd_df = pd.read_csv("twitter-archive-enhanced.csv")


### Download and process associated twitter stats
For having the __Retweet Counts__ and the __Favourite Counts__ for each entry in the __twitter-archive-enhanced.csv__. I will download the whole Twitter API stats by using the Tweet ID. As the Twitter API allows only a certain amount of requests per time it will take a while. Moreover I will collect in the variable __missing__ the Tweet ID's for which it wasn't possible to retrieve any additional information.


In [ ]:
file_name="tweet_json.txt"
missing = []

api = getapi.get_twitter_api()

with open(file_name,mode="w") as file:
    for tid in wrd_df['tweet_id']:
        try:
            output = api.get_status(tid)
        except tweepy.TweepError as e:
            print(str(tid)+":"+str(e))
            missing.append(tid)
        file.write(json.dumps(output._json)+"\n")

888202515573088257:[{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 34


873697596434513921:[{'code': 144, 'message': 'No status found with that ID.'}]
872668790621863937:[{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329:[{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744:[{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320:[{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705:[{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648:[{'code': 144, 'message': 'No status found with that ID.'}]
837012587749474308:[{'code': 144, 'message': 'No status found with that ID.'}]
827228250799742977:[{'code': 144, 'message': 'No status found with that ID.'}]
812747805718642688:[{'code': 144, 'message': 'No status found with that ID.'}]
802247111496568832:[{'code': 144, 'message': 'No status found with that ID.'}]
775096608509886464:[{'code': 144, 'message': 'No status found with that ID.'}]
770743923962707968:[{'code': 144, 'message': 'No sta

Rate limit reached. Sleeping for: 679


In [459]:
tweet_df = pd.DataFrame()


with open("tweet_json.txt","r") as file:
    for index,line in enumerate(file):
        output = json.loads(line)
        tweet_df = tweet_df.append(pd.DataFrame.from_dict(output).head(1))
file.close()

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [461]:
tweet_df = tweet_df.reset_index(drop=True)
tweet_df = tweet_df[['id','retweet_count','favorite_count']]

### Download and process image predictions

In [536]:
pred_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(pred_url)

image_df = pd.read_csv(StringIO(response.text),sep="\t")
image_df.to_csv('image_predictions.tsv',sep='\t')

## Data Assessing
### First View
#### wrd_df

In [171]:
wrd_df.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [160]:
wrd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [179]:
wrd_df[wrd_df.doggo != 'None'].iloc[:5]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None
43,884162670584377345,NaN,NaN,2017-07-09 21:29:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Yogi. He doesn't have any important dog m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/884162670...,12,10,Yogi,doggo,None,None,None
99,872967104147763200,NaN,NaN,2017-06-09 00:02:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a very large dog. He has a date later. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/872967104...,12,10,None,doggo,None,None,None
108,871515927908634625,NaN,NaN,2017-06-04 23:56:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Napolean. He's a Raggedy East Nicaragu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/871515927...,12,10,Napolean,doggo,None,None,None
110,871102520638267392,NaN,NaN,2017-06-03 20:33:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Never doubt a doggo 14/10 https://t.co/AbBLh2FZCH,NaN,NaN,NaN,https://twitter.com/animalcog/status/871075758...,14,10,None,doggo,None,None,None


In [286]:
wrd_df['doggo'].value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

In [289]:
wrd_df['pupper'].value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

In [290]:
wrd_df['floofer'].value_counts()

None       2346
floofer      10
Name: floofer, dtype: int64

In [291]:
wrd_df['puppo'].value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [277]:
wrd_df.groupby(['pupper','floofer','puppo']).doggo.value_counts()

pupper  floofer  puppo  doggo
None    None     None   None     1976
                        doggo      83
                 puppo  None       29
                        doggo       1
        floofer  None   None        9
                        doggo       1
pupper  None     None   None      245
                        doggo      12
Name: doggo, dtype: int64

In [413]:
type(wrd_df['timestamp'][0])

str

In [421]:
wrd_df['tweet_id'].nunique()

2356

In [468]:
wrd_df['name'].value_counts()[:5]

None       745
a           55
Charlie     12
Oliver      11
Lucy        11
Name: name, dtype: int64

#### tweet_df

In [158]:
tweet_df.head(1)

,id,retweet_count,favorite_count
0,892420643555336193,8281,37925


In [161]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
id                2356 non-null int64
retweet_count     2356 non-null int64
favorite_count    2356 non-null int64
dtypes: int64(3)
memory usage: 55.3 KB


#### image_df

In [252]:
image_df.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


In [162]:
image_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [483]:
type(image_df.p1_conf[0])

numpy.float64

In [487]:
wrd_df_clean = wrd_df.copy()
tweet_df_clean = tweet_df.copy()
image_df_clean = image_df.copy()

### Quality Issues
With the overall impression of the assessed data I can identify several quality issues I need to clean for drawing any further conclusions. 

#### WeRateDogs_df (wrd_df)
  1. Remove columns that are unneccesary for further analysis from __wrd_df__.
  2. Remove columns that have almost only null values from __wrd_df__.
  3. Remove rows for which we didn't obtain a twitter status.
  4. Replace four dog type columns into one categorical column in __wrd_df__.
  5. Convert timestamp in __wrd_df__ from string to datetime.
  6. Remove names from __name__ in __wrd_df__ that seems to be unvalid.

#### image_df  
  7. Remove columns that are unneccessary for further analysis from __image_df__.
  8. Remove second __p2__ and third __p3__ estimation from dataframe.

#### tweet_df
  8. Rename Column __id__ to __tweet_id__ for more easier merging.


#### 1 Define
Remove __source__ and __expanded_urls__ from __wrd_df__
  
#### 1 Code

In [488]:
wrd_df_clean.drop(columns=['source','expanded_urls'],inplace=True)

#### 1 Test

In [489]:
wrd_df_clean.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,13,10,Phineas,None,None,None,None


In [490]:
image_df_clean.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


#### 2 Define
Remove
  * in_reply_to_status_id
  * in_reply_to_user_id
  * retweeted_status_id
  * retweeted_status_user_id
  * retweeted_status_time_stamp

from __wrd_df__ as it has almost only null values.

#### 2 Code

In [491]:
wrd_df_clean.drop(columns=['in_reply_to_status_id',
                           'in_reply_to_user_id',
                           'retweeted_status_id',
                           'retweeted_status_user_id',
                           'retweeted_status_timestamp'],inplace=True)

#### 2 Test

In [492]:
wrd_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id              2356 non-null int64
timestamp             2356 non-null object
text                  2356 non-null object
rating_numerator      2356 non-null int64
rating_denominator    2356 non-null int64
name                  2356 non-null object
doggo                 2356 non-null object
floofer               2356 non-null object
pupper                2356 non-null object
puppo                 2356 non-null object
dtypes: int64(3), object(7)
memory usage: 184.1+ KB


#### 3 Define
Remove the rows for the __tweet_id__ we collected in the list __missing__.

#### 3 Code

In [493]:
for tweet_id in missing:
    wrd_df_clean.drop(wrd_df_clean[wrd_df_clean['tweet_id'] == tweet_id].index[0],inplace=True)

#### 3 Test

In [494]:
# if there is removed the right amount of rows, the calculation should result in zero
wrd_df.shape[0] - wrd_df_clean.shape[0] - len(missing)

0

#### 4 Define
Take string values from __doggo, floofer, pupper and puppo__ and put the not __None__ values into one primary categorical column __dogtype__ and taking the risk to remove a secondary label.

#### 4 Code

In [495]:
categories = wrd_df_clean.keys()[-4:].tolist()
categories.append("none")
categories

['doggo', 'floofer', 'pupper', 'puppo', 'none']

In [496]:
wrd_df_clean['dogtype'] = pd.Series(pd.Categorical(values=["none"]*len(wrd_df_clean),categories=categories))


def check_dogtype(df, dogtype, dogtype_string):
    
    mask = dogtype != "None"
    for index,entry in df[mask].iterrows():
        df.loc[index,'dogtype'] = dogtype_string

check_dogtype(wrd_df_clean,wrd_df_clean.doggo,'doggo')            
check_dogtype(wrd_df_clean,wrd_df_clean.pupper,'pupper')
check_dogtype(wrd_df_clean,wrd_df_clean.floofer,'floofer')
check_dogtype(wrd_df_clean,wrd_df_clean.puppo,'puppo')            


wrd_df_clean.drop(columns=['doggo','floofer','pupper','puppo'],inplace=True)

#### 4 Test

In [497]:
wrd_df_clean['dogtype'].value_counts()

none       1946
pupper      256
doggo        82
puppo        30
floofer      10
Name: dogtype, dtype: int64

#### 5 Define
Convert the __timestamp__ column from __wrd_df__ to datetime.

#### 5 Code

In [498]:
wrd_df_clean['timestamp'] = pd.to_datetime(wrd_df_clean['timestamp'])

#### 5 Test

In [499]:
wrd_df_clean['timestamp'][1] - wrd_df_clean['timestamp'][0]

Timedelta('-1 days +07:53:31')

#### 6 Define
Remove names from __name__ in __wrd_df__ that seems to be unvalid like "a".

#### 6 Code

In [500]:
wrd_df_clean.query('name == "a"').apply(lambda x: "None" if x.name == "a" else False)

tweet_id              False
timestamp             False
text                  False
rating_numerator      False
rating_denominator    False
name                  False
dogtype               False
dtype: bool

#### 6 Test

In [501]:
wrd_df_clean.head(1)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dogtype
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,none


#### 7 Define
  Remove __img_num__ from __image_df__.

#### 7 Code

In [502]:
image_df_clean.drop(columns=['img_num'],inplace=True)

#### 7 Test

In [533]:
image_df_clean.head(5)

,tweet_id,jpg_url,p1,p1_conf,p1_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh_springer_spaniel,0.465074,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,redbone,0.506826,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,German_shepherd,0.596461,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,Rhodesian_ridgeback,0.408143,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,miniature_pinscher,0.560311,True


#### 8 Define
Remove __p2__, __p2_dog__, __p2_conf__, __p3__, __p3_dog__ and __p3_conf__ from __image_df__ as it is enough for our purpose to remain the estimation with the highest confidence. 

#### 8 Code

In [504]:
image_df_clean.drop(columns=['p2','p2_dog','p2_conf','p3', 'p3_dog','p3_conf'],inplace=True)

#### 8 Test

In [505]:
image_df_clean.head(1)

,tweet_id,jpg_url,p1,p1_conf,p1_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh_springer_spaniel,0.465074,True


#### 9 Define
Rename Column in __tweet_df__ from __id__ to __tweet_id__.

#### 9 Code

In [506]:
tweet_df_clean.rename(columns={'id':'tweet_id'},inplace=True)

#### 9 Test

In [507]:
tweet_df_clean.head(1)

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8281,37925


### Tidiness Issues
  1. Convert __rating_nominator__ and __rating_denominator__ in __wrd_df__ to a single fraction.
  2. __retweetCount__ and __favouriteCount__ should be merged by __tweet_id__ from __tweet_df__ to __wrd_df__.


#### 1 Define
Convert __rating_numerator__ and __rating_denominator__ in __wrd_df__ to a single fraction __rating__ and remove them.

#### 1 Code

In [508]:
wrd_df_clean['rating'] = wrd_df_clean['rating_numerator'] / wrd_df_clean['rating_denominator']
wrd_df_clean.drop(columns=['rating_numerator','rating_denominator'],inplace=True)

#### 1 Test

In [509]:
wrd_df_clean.head(1)

,tweet_id,timestamp,text,name,dogtype,rating
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,Phineas,none,1.3


#### 2 Define
Merge wrd_df with tweet_df by using the __tweet_id__ as the key. 

#### 2 Code

In [510]:
twitter_archive_master = wrd_df_clean.merge(tweet_df_clean,how='outer',left_on='tweet_id',right_on='tweet_id')

#### 2 Test

In [511]:
twitter_archive_master.head(5)

,tweet_id,timestamp,text,name,dogtype,rating,retweet_count,favorite_count
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,Phineas,none,1.3,8281,37925
1,892177421306343426,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,Tilly,none,1.3,6117,32566
2,891815181378084864,2017-07-31 00:18:03,This is Archie. He is a rare Norwegian Pouncin...,Archie,none,1.2,4051,24522
3,891689557279858688,2017-07-30 15:58:51,This is Darla. She commenced a snooze mid meal...,Darla,none,1.3,8422,41271
4,891327558926688256,2017-07-29 16:00:24,This is Franklin. He would like you to stop ca...,Franklin,none,1.2,9122,39452


### Write finished dataframes to csv

In [535]:
twitter_archive_master.to_csv('twitter_archive_master.csv',sep=',',index=False)
image_df_clean.to_csv('twitter_image_prediction.csv',sep=',',index=False)